In [19]:
import json

In [24]:
file = open('../Datasets/RamayanaEngSen.json')

In [25]:
data = json.load(file)

In [6]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2022-04-26 23:27:13 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-04-26 23:27:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-04-26 23:27:14 INFO: Use device: cpu
2022-04-26 23:27:14 INFO: Loading: tokenize
2022-04-26 23:27:14 INFO: Loading: pos
2022-04-26 23:27:14 INFO: Loading: lemma
2022-04-26 23:27:14 INFO: Loading: depparse
2022-04-26 23:27:14 INFO: Done loading processors!


In [28]:
def get_entities(sent):
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""
    nl = nlp(sent)
    for sen in nl.sentences:
        for word in sen.words:
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
            if word.deprel != "punct":
      # check: token is a compound word or not
                if word.deprel == "compound":
                    prefix = word.text
                # if the previous word was also a 'compound' then add the current word to it
                    if prv_tok_dep == "compound":
                        prefix = prv_tok_text + " "+ word.text
                # check: token is a modifier or not
                if word.deprel.endswith("mod") == True:
                    modifier = word.text
        # if the previous word was also a 'compound' then add the current word to it
                    if prv_tok_dep == "compound":
                        modifier = prv_tok_text + " "+ word.text

      ## chunk 3
                if word.deprel.find("subj") == True:
                    ent1 = modifier +" "+ prefix + " "+ word.text
                    prefix = ""
                    modifier = ""
                    prv_tok_dep = ""
                    prv_tok_text = ""      

      ## chunk 4
                if word.deprel.find("obj") == 0:
                    ent2 = modifier +" "+ prefix +" "+ word.text

      ## chunk 5  
      # update variables
                prv_tok_dep = word.deprel
                prv_tok_text = word.text
  #############################################################
    return [ent1.strip(), ent2.strip()]

In [29]:
entity_pairs = []
for i in data[:50]:
    entity_pairs.append(get_entities(nlp(i)))

In [30]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span
def get_relation(sent):

    doc = nlp(sent)
    verb = ''
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos == 'VERB' and word.deprel == 'root':
                verb = word.text
            if len(verb) == 0 and word.upos == 'VERB':
                verb = word.text
        return verb

In [31]:
relations = []
for sent in data[:50]:
    relations.append(get_relation(sent))

In [48]:
import pandas as pd
source = [i[0] for i in entity_pairs]
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})

In [49]:
import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

In [50]:
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()